In [1]:
import sys
import dgl
import dgl.function as fn
import os
import multiprocessing as mp
from tqdm import tqdm
import pdb
import numpy as np
import torch
import torch.nn as nn
import logging
from utils.parser import parse_args
from utils.dataloader import Dataloader
from utils.utils import config, construct_negative_graph, choose_model, load_mf_model, NegativeGraph
from utils.tester import Tester
from models.sampler import NegativeSampler
import wandb
import math

FileNotFoundError: Could not find module 'd:\environment\conda-c\envs\PyTorch\lib\site-packages\dgl\dgl.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [20]:
numbers=[8,4,2,2,2,2,1,1]
square_sum=math.sqrt(sum(i**2 for i in numbers))
numbers = [x / square_sum for x in numbers]

ma=1.2*max(numbers)
mi=0.3*min(numbers)

result=[(x-mi) /(ma-mi) for x in numbers]


result = [round(x / sum(result),2) for x in result]
print(result)

[0.39, 0.19, 0.09, 0.09, 0.09, 0.09, 0.04, 0.04]


In [16]:
import math
numbers=[8,4,2,2,2,2,1,1]
square_sum=math.sqrt(sum(i**2 for i in numbers))
numbers = [math.exp(x/square_sum) for x in numbers]
ma=max(numbers)
mi=0.4*min(numbers)
result=[(x-mi) /(ma-mi) for x in numbers]
print(result)


[1.0, 0.585939221919969, 0.43381058801199746, 0.43381058801199746, 0.43381058801199746, 0.43381058801199746, 0.3685243405799671, 0.3685243405799671]


In [65]:
result

[4, 2, 2, 2, 2, 2, 1, 1]

In [2]:
def pairwise_cosine_similarity(x, y, zero_diagonal: bool = False):
    r"""
    Calculates the pairwise cosine similarity matrix

    Args:
        x: tensor of shape ``(batch_size, M, d)``
        y: tensor of shape ``(batch_size, N, d)``
        zero_diagonal: determines if the diagonal of the distance matrix should be set to zero

    Returns:
        A tensor of shape ``(batch_size, M, N)``
    """
    x_norm = torch.linalg.norm(x, dim=2, keepdim=True)
    y_norm = torch.linalg.norm(y, dim=2, keepdim=True)
    distance = torch.matmul(torch.div(x, x_norm), torch.div(y, y_norm).permute(0, 2, 1))
    

    return distance

In [3]:
import torch
x=torch.randn(23,48,32)
y=x
z=pairwise_cosine_similarity(x,y)

In [8]:
print(z)
print(z.mean(dim=1))

tensor([[[ 1.0000, -0.0261,  0.0632, -0.1281],
         [-0.0261,  1.0000,  0.2065, -0.0501],
         [ 0.0632,  0.2065,  1.0000,  0.1793],
         [-0.1281, -0.0501,  0.1793,  1.0000]],

        [[ 1.0000,  0.1538, -0.0039,  0.2631],
         [ 0.1538,  1.0000,  0.2228,  0.0622],
         [-0.0039,  0.2228,  1.0000,  0.1332],
         [ 0.2631,  0.0622,  0.1332,  1.0000]]])
tensor([[0.2273, 0.2826, 0.3623, 0.2503],
        [0.3532, 0.3597, 0.3380, 0.3646]])


In [16]:
print(z.shape)
print(z.mean(dim=2).mean(dim=1).unsqueeze(dim=1))
print(z.mean(dim=2))

torch.Size([2, 4, 4])
tensor([[0.1581],
        [0.2868]])
tensor([[0.1371, 0.1553, 0.1830, 0.1571],
        [0.2358, 0.3025, 0.3012, 0.3075]])


In [9]:
original_list = [42, 17, 8, 99, 23]
sorted_indices = sorted(range(len(original_list)), key=lambda i: original_list[i], reverse=True)
sorted_list = [original_list[i] for i in sorted_indices]

print("原始列表:", original_list)
print("排序后的列表:", sorted_list)
print("排序后的索引:", sorted_indices)


原始列表: [42, 17, 8, 99, 23]
排序后的列表: [99, 42, 23, 17, 8]
排序后的索引: [3, 0, 4, 1, 2]
